In [328]:
import pandas as pd
import numpy as np

df = pd.read_csv('WineDataset.csv')
df = df.sample(frac=1, random_state=42).reset_index(drop=True)


if df.isnull().sum().sum() > 0:
    df = df.fillna(df.mean()) 

# отделение от целевой переменной
X = df.drop('Wine', axis=1).values  
y = df['Wine'].values 

# нормалзация
X = (X - np.min(X, axis=0)) / (np.max(X, axis=0) - np.min(X, axis=0))



# обуч и тест выборки
train_size = int(0.72 * X.shape[0])

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [329]:
class KNN:
    def __init__(self, k=3):
        self.k = k
    
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)
    
    def _predict(self, x):
        #  евклидово расстояние до всех обучающих примеров
        distances = np.linalg.norm(self.X_train - x, axis=1)
        # индексы k ближ соседей
        k_indices = np.argsort(distances)[:self.k]
        # метки классов ближ соседей
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        # наиболее часто встречающийся класс
        most_common = np.bincount(k_nearest_labels).argmax()
        return most_common

def confusion_matrix_metrics(y_true, y_pred, labels):
    confusion_metrics = {}
    for label in labels:
        TP = np.sum((y_true == label) & (y_pred == label))  # Истинные положительные
        TN = np.sum((y_true != label) & (y_pred != label))  # Истинные отрицательные
        FP = np.sum((y_true != label) & (y_pred == label))  # Ложные положительные
        FN = np.sum((y_true == label) & (y_pred != label))  # Ложные отрицательные

        confusion_metrics[label] = {'TP': TP, 'TN': TN, 'FP': FP, 'FN': FN}
    
    return confusion_metrics

def print_confusion_matrix(conf_matrix):
    print("Матрица ошибок:")
    for label, metrics in conf_matrix.items():
        print(f"Класс {label}:")
        print(f"  Истинно положительные (TP): {metrics['TP']}")
        print(f"  Истинно отрицательные (TN): {metrics['TN']}")
        print(f"  Ложно положительные (FP): {metrics['FP']}")
        print(f"  Ложно отрицательные (FN): {metrics['FN']}")



In [330]:
# Случайный выбор 5 признаков
np.random.seed(42)
random_features = np.random.choice(df.columns[:-1], 5, replace=False)
random_indices = [df.columns.get_loc(feature) for feature in random_features]  # получаем индексы случайных признаков

X_train_random = X[:train_size, random_indices]
X_test_random = X[train_size:, random_indices]


# Модель 1: k-NN с случайно выбранными признаками
knn_model_1 = KNN(k=3)
knn_model_1.fit(X_train_random, y_train)
y_pred_1 = knn_model_1.predict(X_test_random)

# Оценка для случайных признаков
labels = np.unique(y_test)
conf_matrix_model_1 = confusion_matrix_metrics(y_test, y_pred_1, labels)

print(f"Матрица ошибок для модели с случайными признаками (k=3):")
print_confusion_matrix(conf_matrix_model_1)


Матрица ошибок для модели с случайными признаками (k=3):
Матрица ошибок:
Класс 1:
  Истинно положительные (TP): 12
  Истинно отрицательные (TN): 35
  Ложно положительные (FP): 3
  Ложно отрицательные (FN): 0
Класс 2:
  Истинно положительные (TP): 19
  Истинно отрицательные (TN): 27
  Ложно положительные (FP): 0
  Ложно отрицательные (FN): 4
Класс 3:
  Истинно положительные (TP): 14
  Истинно отрицательные (TN): 33
  Ложно положительные (FP): 2
  Ложно отрицательные (FN): 1


In [331]:
# Фиксированный набор 
fixed_features = ['Alcohol', 'Malic Acid', 'Ash']
fixed_indices = [df.columns.get_loc(feature) for feature in fixed_features]


X_train_fixed = X[:train_size, fixed_indices]
X_test_fixed = X[train_size:, fixed_indices]

# Модель 2: k-NN с фиксированными признаками
knn_model_2 = KNN(k=3)
knn_model_2.fit(X_train_fixed, y_train)
y_pred_2 = knn_model_2.predict(X_test_fixed)

conf_matrix_model_2 = confusion_matrix_metrics(y_test, y_pred_2, labels)

print(f"Матрица ошибок для модели с фиксированными признаками (k=3):")
print_confusion_matrix(conf_matrix_model_2)


Матрица ошибок для модели с фиксированными признаками (k=3):
Матрица ошибок:
Класс 1:
  Истинно положительные (TP): 12
  Истинно отрицательные (TN): 35
  Ложно положительные (FP): 3
  Ложно отрицательные (FN): 0
Класс 2:
  Истинно положительные (TP): 20
  Истинно отрицательные (TN): 23
  Ложно положительные (FP): 4
  Ложно отрицательные (FN): 3
Класс 3:
  Истинно положительные (TP): 10
  Истинно отрицательные (TN): 34
  Ложно положительные (FP): 1
  Ложно отрицательные (FN): 5


In [332]:
def evaluate_knn(k_values, X_train, X_test, y_train, y_test, labels):
    for k in k_values:
        knn = KNN(k=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        
        conf_matrix = confusion_matrix_metrics(y_test, y_pred, labels)
        print(f"Матрица ошибок для k={k}:")
        print_confusion_matrix(conf_matrix)


        

k_values = [3, 5, 10]




In [333]:
print("Оценка для модели с случайными признаками:")
evaluate_knn(k_values, X_train_random, X_test_random, y_train, y_test, labels)

Оценка для модели с случайными признаками:
Матрица ошибок для k=3:
Матрица ошибок:
Класс 1:
  Истинно положительные (TP): 12
  Истинно отрицательные (TN): 35
  Ложно положительные (FP): 3
  Ложно отрицательные (FN): 0
Класс 2:
  Истинно положительные (TP): 19
  Истинно отрицательные (TN): 27
  Ложно положительные (FP): 0
  Ложно отрицательные (FN): 4
Класс 3:
  Истинно положительные (TP): 14
  Истинно отрицательные (TN): 33
  Ложно положительные (FP): 2
  Ложно отрицательные (FN): 1
Матрица ошибок для k=5:
Матрица ошибок:
Класс 1:
  Истинно положительные (TP): 12
  Истинно отрицательные (TN): 36
  Ложно положительные (FP): 2
  Ложно отрицательные (FN): 0
Класс 2:
  Истинно положительные (TP): 19
  Истинно отрицательные (TN): 26
  Ложно положительные (FP): 1
  Ложно отрицательные (FN): 4
Класс 3:
  Истинно положительные (TP): 14
  Истинно отрицательные (TN): 33
  Ложно положительные (FP): 2
  Ложно отрицательные (FN): 1
Матрица ошибок для k=10:
Матрица ошибок:
Класс 1:
  Истинно положит

In [334]:
print("Оценка для модели с фиксированными признаками:")
evaluate_knn(k_values, X_train_fixed, X_test_fixed, y_train, y_test, labels)

Оценка для модели с фиксированными признаками:
Матрица ошибок для k=3:
Матрица ошибок:
Класс 1:
  Истинно положительные (TP): 12
  Истинно отрицательные (TN): 35
  Ложно положительные (FP): 3
  Ложно отрицательные (FN): 0
Класс 2:
  Истинно положительные (TP): 20
  Истинно отрицательные (TN): 23
  Ложно положительные (FP): 4
  Ложно отрицательные (FN): 3
Класс 3:
  Истинно положительные (TP): 10
  Истинно отрицательные (TN): 34
  Ложно положительные (FP): 1
  Ложно отрицательные (FN): 5
Матрица ошибок для k=5:
Матрица ошибок:
Класс 1:
  Истинно положительные (TP): 10
  Истинно отрицательные (TN): 36
  Ложно положительные (FP): 2
  Ложно отрицательные (FN): 2
Класс 2:
  Истинно положительные (TP): 20
  Истинно отрицательные (TN): 21
  Ложно положительные (FP): 6
  Ложно отрицательные (FN): 3
Класс 3:
  Истинно положительные (TP): 10
  Истинно отрицательные (TN): 33
  Ложно положительные (FP): 2
  Ложно отрицательные (FN): 5
Матрица ошибок для k=10:
Матрица ошибок:
Класс 1:
  Истинно пол